# Growing Subsequences
Given an array of numbers $v_0, v_1, \ldots, v_{n-1}$, find the maximum integer $k$ for which there exists a strictly increasing sequence of indices
$
0 \;\le\; \pi_1 \;<\; \pi_2 \;<\;\dots\;<\;\pi_k \;\le\; n-1
$
satisfying
$
v_{\pi_1} \;\le\; v_{\pi_2} \;\le\;\dots\;\le\; v_{\pi_k}.
$

In [12]:
using BenchmarkTools

# Test
v = rand(-30:30,28) 

28-element Vector{Int64}:
 -11
  19
   3
 -13
  17
  -5
 -26
 -26
  -5
 -24
   ⋮
 -17
   7
  -1
 -30
 -10
 -11
   5
   5
 -28

In [13]:
# ChatGPT Generated
function brute_force_lnnds_length(v)
    n = length(v)
    best_len = 0
    
    # Check all subsets via bitmask from 0 to 2^n - 1
    for mask in 0:(1 << n) - 1
        current_len = 0
        last_val = typemin(Int)
        valid = true
        
        for i in 0:(n-1)
            if (mask & (1 << i)) != 0
                if v[i+1] >= last_val
                    # Extend this subsequence
                    current_len += 1
                    last_val = v[i+1]
                else
                    # This subset fails the non-decreasing condition
                    valid = false
                    break
                end
            end
        end
        
        # Update best length
        if valid && current_len > best_len
            best_len = current_len
        end
    end
    
    return best_len
end

# Example usage:
max_length = @btime brute_force_lnnds_length(v)
println("Length of the longest non-decreasing subsequence is: ", max_length)


  1.702 s (0 allocations: 0 bytes)
Length of the longest non-decreasing subsequence is: 9


# Solution

In [14]:
function sub(v::Vector{Int}, last::Bool=false)
    # Base case: if there's only one element, return it as the subsequence.
    if length(v) == 1
        return v
    end

    best = Int[]  

    for i in 1:(length(v)-1)
        subseq = sub(v[1:i], true)
        
        if last
            grow_ending = [el for el in subseq if el <= v[end]]
            push!(grow_ending, v[end])
            
            if length(grow_ending) > length(best)
                best = grow_ending
            end
        else
            if length(subseq) > length(best) ||
               (length(subseq) == length(best) && (isempty(best) || subseq[end] < best[end]))
                best = subseq
            end
        end
    end

    if !last && (isempty(best) || v[end] >= best[end])
        push!(best, v[end])
    end

    return best
end

# Example usage:
println("Result: ", @btime sub(v))


  8.810 s (702675531 allocations: 30.27 GiB)
Result: [-26, -26, -24, -9, -7, -7, 4, 5, 5]


# Memo Version

In [15]:
function sub_memo(v::Vector{Int})
    cache = Dict{Tuple{Tuple{Int,Vararg{Int}}, Bool}, Vector{Int}}()
    
    function _sub(v::Vector{Int}, last::Bool=false)
        key = (tuple(v...), last)

        if haskey(cache, key)
            return cache[key]
        end

        if length(v) == 1
            cache[key] = v
            return v
        end

        best = Int[]
        
        for i in 1:(length(v)-1)
            subseq = _sub(v[1:i], true)
            
            if last
                grow_ending = [el for el in subseq if el <= v[end]]
                push!(grow_ending, v[end])
                
                if length(grow_ending) > length(best)
                    best = grow_ending
                end
            else
                if length(subseq) > length(best) ||
                   (length(subseq) == length(best) && (isempty(best) || subseq[end] < best[end]))
                    best = subseq
                end
            end
        end

        if !last && (isempty(best) || v[end] >= best[end])
            push!(best, v[end])
        end
        
        cache[key] = best
        return best
    end

    return _sub(v)
end

# Example usage:
println("Result: ", @btime sub_memo(v))


  413.250 μs (3644 allocations: 207.56 KiB)
Result: [-26, -26, -24, -9, -7, -7, 4, 5, 5]


# Testing

In [16]:
for size in 2:20
    for _ in 1:10
        v = rand(-10:10, size)
        l_b = brute_force_lnnds_length(v)
        l_l = length(sub(v))

        if l_b != l_l
            println("\n----")
            println("$v")
            println("$l_b")
            println("$l_l")
            println("----")
        end
    end
end